```{contents}
```
## agent_scratchpad 


`agent_scratchpad` is a **temporary reasoning workspace** used by LangChain agents to **store intermediate steps** such as:

* Tool calls made so far
* Tool outputs (observations)
* Partial reasoning state required to decide the next step

> It is **not user input** and **not final output**.
> It exists only to help the agent reason across steps.

---

### Why `agent_scratchpad` Exists

Agents operate iteratively. They must remember:

* Which tools were already called
* What results were returned
* What still needs to be done

Without `agent_scratchpad`, the agent would:

* Repeat tool calls
* Lose track of progress
* Fail to converge on an answer

---

### Where `agent_scratchpad` Fits

```
User Input
   ↓
Agent Prompt
   ↓
agent_scratchpad  ← (intermediate steps injected here)
   ↓
LLM decides next action
   ↓
Tool Call / Final Answer
```

The scratchpad is **continuously updated** during execution.

---

### What Goes Into `agent_scratchpad`

Typically contains:

* Previous tool calls
* Tool results
* Structured intermediate steps

Conceptually:

```
Action: ticket_count(source="jira")
Observation: 128
Action: sla_threshold()
Observation: 100
```

The agent reads this to decide the next step.

---

### `agent_scratchpad` vs Chat History

| Aspect        | agent_scratchpad | Chat History |
| ------------- | ---------------- | ------------ |
| Scope         | Internal         | User-visible |
| Purpose       | Reasoning        | Conversation |
| Persistence   | Single run       | Multi-turn   |
| Shown to user | ❌                | ✅            |

They serve **completely different roles**.

---

### How LangChain Uses `agent_scratchpad`

LangChain automatically:

1. Collects intermediate steps
2. Formats them
3. Injects them into `{agent_scratchpad}`

You usually **do not populate it manually**.

---

### Prompt Usage

#### ChatPromptTemplate Example

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant. Use tools if needed."),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}")
])
```

Key rule:

> `agent_scratchpad` **must exist** in agent prompts that support tools.

---

#### What Happens If You Omit It

If `{agent_scratchpad}` is missing:

* Tool calls may fail
* Agent may loop
* Errors like `KeyError: 'intermediate_steps'` appear

This happens because the agent cannot inject prior steps.

---

### `agent_scratchpad` and `intermediate_steps`

Internally:

* LangChain stores tool calls in `intermediate_steps`
* Converts them into a formatted `agent_scratchpad`

Relationship:

```
intermediate_steps → formatted → agent_scratchpad
```

You saw this error earlier because the prompt expected one but the runtime did not provide it correctly.

---

### Visibility and Safety

#### Is `agent_scratchpad` shown to users?

❌ No (unless you explicitly log it)

#### Should it be logged?

✅ Yes (for debugging)

#### Should it be stored long-term?

❌ No (can contain sensitive data)

---

### ReAct Agents and `agent_scratchpad`

ReAct agents rely heavily on the scratchpad:

```
Thought
Action
Observation
Thought
```

These steps are accumulated inside `agent_scratchpad`.

In production:

* Thoughts are hidden
* Only final answer is returned

---

### OpenAI Tools Agent and `agent_scratchpad`

* Still used internally
* Tool calls are structured (JSON)
* Scratchpad exists but is less visible

You still must include `{agent_scratchpad}` in the prompt.

---

### Common Mistakes

#### Manually passing `agent_scratchpad`

❌ Never do this

#### Using it in non-agent chains

❌ Not applicable

#### Showing it to users

❌ Security and UX issue

#### Removing it to “simplify”

❌ Agent breaks

---

### Best Practices

* Always include `{agent_scratchpad}` in agent prompts
* Never expose it in UI
* Log it only in debug mode
* Let LangChain manage it
* Keep prompts minimal around it

---

### Interview-Ready Summary

> “`agent_scratchpad` is an internal prompt variable that stores an agent’s intermediate reasoning steps and tool interactions, enabling multi-step decision-making. It is essential for agent execution but is hidden from users.”

---

### Rule of Thumb

* **Chains → no scratchpad**
* **Agents → scratchpad required**
* **Users → never see it**
* **Debugging → inspect it**